# Cohort: # 04

# Team No.: # 10

## Members:
* Loh De Rong (1003557)
* Hadi Alwi (1003482)
* Ishika Kajaria (1003555)
* Pham Trung Viet (1003589)

In [1]:
import random
import math

# Part 1 - Getting the Cost Function Ready

### Task 1

Instance variables are being instantiated. The init method initializes all parameters to zero. All penalty parameters are set to a default value of 10.

In [2]:
class PrimerDesign(object):
    
    def __init__ (self, name):
        
        '''parameters for the length criterion'''
        self.max_length = 0
        self.min_length = 0
        self.penalty_length = 10
        
        '''parameters for the temperature difference criterion'''
        self.max_tdiff = 0
        self.min_tdiff = 0
        self.penalty_tdiff = 10
        
        '''parameters for the cg content criterion'''
        self.max_cg = 0
        self.min_cg = 0
        self.penalty_cg = 10
        
        '''parameters for the annealing temperature criterion'''
        self.max_temp = 0
        self.min_temp = 0
        self.penalty_temp = 10
        
        '''parameters for the run criterion'''
        self.run_threshold = 0
        self.penalty_runs = 10
        
        '''parameters for the repeat criterion'''
        self.repeat_threshold = 0
        self.penalty_repeats = 10
        
        '''parameters for the specificity criterion'''
        self.penalty_specificity = 10 
        
        '''locations where the forward primer should be chosen from'''
        self.fp_start = 0
        self.fp_end = 0
        
        '''locations where the reverse primer should be chosen from'''
        self.rp_start = 0
        self.rp_end = 0
        
        ''' parameters for the simulated annealing portion'''
        self.initial_temperature = 200
        self.stopping_temperature = 0.01
        self.drop_fraction = 0.999
        

### Task 2

The set_dna_sequence method takes in a raw gene sequence, removes the numbers and spaces, and returns only a, t, c, g in lower case characters. The gene will be considered as invalid if it is not in a string format, contains upper case characters, or characters that are not a, t, c, g.

In [3]:
class PrimerDesign(PrimerDesign): 
    
    def set_dna_sequence(self, dna_sequence):
        
        # check if dna_sequence is a string
        if not isinstance(dna_sequence, str):
            return "ERROR"
        
        dna_sequence = dna_sequence.replace(" ", "") # removing white spaces in the string
        dna_sequence = dna_sequence.replace("\n", "") # removing new lines in the string
        edited_dna_sequence = ""
        
        for base in dna_sequence:
            if base in "atcg":
                edited_dna_sequence += base # adding base
            elif base not in ['0','1','2','3','4','5','6','7','8','9']:
                return "ERROR" # check that base is not any other alphabet besides a,t,c,g
        
        # initializes dna_sequence if all conditions met
        self.dna_sequence = edited_dna_sequence

In [4]:
'''Testing method in Task 2'''

# Test Case 1: check that the dna_sequence containing only a,t,c,g in lower case is instantiated correctly
DNA2_1 = PrimerDesign("DNA2_1")
DNA2_1.set_dna_sequence("atcgcgatcg")
assert DNA2_1.dna_sequence == "atcgcgatcg"

# Test Case 2: check that the dna_sequence is stripped off its numbers, newlines and spaces, and instantiated correctly
DNA2_2 = PrimerDesign("DNA2_2")
DNA2_2.set_dna_sequence("1 atc\n 2atac\ngt  3tag 4ccgg ")
assert DNA2_2.dna_sequence == "atcatacgttagccgg"

# Test Case 3: check that the dna_sequence is of type str
DNA2_3 = PrimerDesign("DNA2_3")
assert DNA2_3.set_dna_sequence(["this does not work"]) == "ERROR"

# Test Case 4: check that the dna_sequence only contains a,t,c,g in lower case
DNA2_4 = PrimerDesign("DNA2_4")
assert DNA2_4.set_dna_sequence("aTcggggatC") == "ERROR"

# Test Case 5: check that the dna_sequence does not contain other alphabets besides a,t,c,g
DNA2_5 = PrimerDesign("DNA2_4")
assert DNA2_5.set_dna_sequence("kcggtfaggat") == "ERROR"

### Task 3

The method func_select_random aims to select a forward primer or reverse primer at random with a specified length.

In [5]:
class PrimerDesign(PrimerDesign):
    
    def func_select_random(self, sqtype='forward', length = 20):
        
        # primer should be either forward or reverse
        # the length has to be a positive number
        if (sqtype != 'forward' and sqtype != 'reverse') or (length < 0 or length > len(self.dna_sequence)):
            return "ERROR"
        
        # returns empty string if length = 0
        if length == 0:
            return ""
        
        elif(sqtype == 'forward'):
            start_limit = self.fp_start 
            end_limit = self.fp_end 
            
        elif(sqtype == 'reverse'):
        
            start_limit = self.rp_start 
            end_limit = self.rp_end
            
        # generate a random starting position that lies between these limits
        starting_point = random.randrange(start_limit, end_limit - length + 1)
        
        # adding bases to obtain the full-length primer
        primer = ""
        for i in range(length):
            primer += self.dna_sequence[starting_point + i]
        
        return primer

In [6]:
'''Testing method in Task 3'''

DNA3 = PrimerDesign("DNA3")
DNA3.set_dna_sequence("aaaaaaaaaattttttttttccccccccccggggggggggaaaaaaaaaattttttttttccccccccccgggggggggg")
DNA3.fp_start = 5
DNA3.fp_end = 35
DNA3.rp_start = 50
DNA3.rp_end = 80

# Test Case 1: check that sqtype is either forward or reverse
assert DNA3.func_select_random(sqtype = "thisdoesnotwork") == "ERROR"

# Test Case 2: check that length > 0
assert DNA3.func_select_random(length = -5) == "ERROR"

# Test Case 3: check that length <= dna_sequence
assert DNA3.func_select_random(length = 2000) == "ERROR"

# Test Case 4: check that a random forward primer sequence is selected within limits
assert DNA3.func_select_random(sqtype='forward', length = 20) in "aaaaattttttttttccccccccccggggg"

# Test Case 5: check that a random reverse primer sequence is selected within limits
assert DNA3.func_select_random(sqtype='reverse', length = 20) in "ttttttttttccccccccccgggggggggg"

### Task 4

The following methods calculate the properties for a given primer string. We assume that the primer sequence is valid because it will be called using the func_select_random method. Also note that there is no need to reverse complement to find the reverse primer because the cost functions will effectively give the same value e.g. the cg fraction will be the same whether you reverse complement it or not. 

In [7]:
class PrimerDesign(PrimerDesign): 
    
    # calculates the functional length of the primer sequence
    def func_length(self, sq):
        return len(sq)
    
    # calculates the fraction of cg bases in the primer sequence
    def func_cg_fraction(self, sq):
        if len(sq) == 0:
            return 0
        return (sq.count("c") + sq.count("g"))/len(sq)
    
    # calculates the annealing temperature of the primer
    def func_temperature(self,sq):
        return 4*(sq.count("c") + sq.count("g")) + 2*(sq.count("a") + sq.count("t"))           

In [8]:
class PrimerDesign(PrimerDesign):

    # calculates the number of base runs that exceed the acceptable threshold
    def func_count_runs(self,sq):
        R_max = self.run_threshold
        n_runs = 0
        count = 1
        
        for i in range(len(sq)-1):
            # adds 1 to count if the next element is the same as the previous
            if sq[i] == sq[i+1]:
                count += 1
            else:
                # adds 1 to base run if the cumulative count exceeds threshold
                # then reset the counter for the next base
                if count > R_max:
                    n_runs += 1
                count = 1 # counter reset for the next base
        return n_runs

In [9]:
class PrimerDesign(PrimerDesign):
    
    # calculates the number of repeats detected
    # repeats refer to repeating sequences of two bases in the primer
    def func_count_repeats(self,sq):
        
        di_repeats = ['at','ac','ag','ca','ct','cg','ga','gt','gc','ta','tc','tg']
        count = 0
        
        for i in range(0, len(sq)-2, 2):
            # adds 1 to count if the next sequence is a repeat of the previous
            # starting from the 0th element
            current_base1 = sq[i:i+2]
            next_base1 = sq[i+2:i+4]
            if next_base1 == current_base1 and current_base1 in di_repeats:
                count += 1
            
            # adds 1 to count if the next sequence is a repeat of the previous
            # starting from the 1st element
            current_base2 = sq[i+1:i+3]
            next_base2 = sq[i+3:i+5]
            if next_base2 == current_base2 and current_base2 in di_repeats:
                count += 1    
        return count

In [10]:
class PrimerDesign(PrimerDesign):
    def func_count_specificity(self,sq):
        n_positions = 0
       
        #iterate through dna_sequence and count the number of primer occurences
        for i in range(len(self.dna_sequence) - len(sq) +1):
            if sq == self.dna_sequence[i:i+len(sq)]:
                n_positions += 1
        return n_positions

## Test Cases for 15 test cases are provided here.

## (from eDimension)
### case 1. 'cattaaaaatacgaaaaaagtcat'
It has length 24, cg fraction of 5/24, temperature of 4(5) + 2(19) = 58, two runs of a no repeats, hence a repeat score of 0.

### case 2. 'atatatatattttatatataa'
It has length 21, cg fraction of 0/21, temperature of 4(0) + 2(21) = 42, no runs of a or t. For repeats, five instances of at, four instances of ta, followed by four instances of ta and three instances of at. Thus the repeat score is 12.

### case 3 - edge case '' (empty string).
It has length 0, cg fraction 0, temperature 0, no runs, no repeats.

## (own test case)
### case 4. 'aattttggccccctgact'
It has length 20, cg fraction 10/20, temperature of 4(10) + 2(10) = 60, 1 run of c. For repeats, three instances of ga and two instances of ag. Thus the repeat score is 3.

In [11]:
'''Testing methods in Task 4'''

DNA4 = PrimerDesign("DNA4")
DNA4.run_threshold = 4
sq1 = 'cattaaaaatacgaaaaaagtcat'
sq2 = 'atatatatattttatatataa'
sq3 = ''
sq4 = 'aattttggccccctgagaga'

# Test Cases 1-4: check length
assert DNA4.func_length(sq1) == 24
assert DNA4.func_length(sq2) == 21
assert DNA4.func_length(sq3) == 0
assert DNA4.func_length(sq4) == 20

# Test Cases 5-8: check cg fraction
assert DNA4.func_cg_fraction(sq1) == 5/24
assert DNA4.func_cg_fraction(sq2) == 0/21
assert DNA4.func_cg_fraction(sq3) == 0
assert DNA4.func_cg_fraction(sq4) == 10/20

# Test Cases 9-12: check annealing temperature
assert DNA4.func_temperature(sq1) == 58
assert DNA4.func_temperature(sq2) == 42
assert DNA4.func_temperature(sq3) == 0
assert DNA4.func_temperature(sq4) == 60

# Test Cases 13-16: check runs
assert DNA4.func_count_runs(sq1) == 2
assert DNA4.func_count_runs(sq2) == 0
assert DNA4.func_count_runs(sq3) == 0
assert DNA4.func_count_runs(sq4) == 1

# Test Cases 17-20: check repeats
assert DNA4.func_count_repeats(sq1) == 0
assert DNA4.func_count_repeats(sq2) == 12
assert DNA4.func_count_repeats(sq3) == 0
assert DNA4.func_count_repeats(sq4) == 3

### Task 6

The following methods implement the cost functions for each individual criterion. Penalties will be applied to the properties that do not fall within the acceptable limits, according to the mathematical expressions given in the handout.

The penalty parameter 𝛼 is an adjustable value to be decided. A larger penalty value penalizes deviations from the acceptable limits more. Every cost function will have its own penalty parameter.

In [12]:
class PrimerDesign(PrimerDesign):
    
    '''Primer length is usually between 18 and 22 nucleotides'''
    def cost_length(self, sq):
        
        # calculate the primer length
        sq_len = len(sq)
        
        # penalization if primer length > max length
        if(sq_len > self.max_length): 
            return (sq_len - self.max_length)*self.penalty_length
        
        # no penalization if min length <= primer length <= max length
        elif(sq_len >= self.min_length): 
            return 0
        
        # penalization if primer length < min length
        else:
            return (self.min_length - sq_len)*self.penalty_length
    
    '''The primer annealing temperature is usually between 55°C and 63°C'''
    def cost_temperature(self, sq):
        
        # calculate the primer annealing temp
        temp = self.func_temperature(sq)
        
        # penalization if primer annealing temp > max temp
        if(temp > self.max_temp):
            return (temp - self.max_temp)*self.penalty_temp
        
        # no penalization if min temp <= primer annealing temp <= max temp
        elif(temp >= self.min_temp):
            return 0
        
        # penalization if primer annealing temp < min temp
        else:
            return (self.min_temp - temp)*self.penalty_temp 
    
    '''The CG content is usually between 40% and 60%'''
    def cost_cgcontent(self,sq):
        
        # calculate the primer % cg content
        cg_content = self.func_cg_fraction(sq)
        
        # penalization if primer % cg content > max % cg content
        if(cg_content > self.max_cg):
            return (cg_content - self.max_cg)*self.penalty_cg
        
        # no penalization if min % cg content <= primer % cg content <= max % cg content
        elif(cg_content >= self.min_cg):
            return 0
        
        # penalization if % cg content < min % cg content
        else:
            return (self.min_cg - cg_content)*self.penalty_cg
        
    '''The difference in annealing temperatures between forward and reverse primers is usually less than 2°C'''
    def cost_temperature_difference(self, fp, rp):
        
        # calculate the difference in the two primers' annealing temp
        temp_diff = abs(self.func_temperature(fp) - self.func_temperature(rp)) # always a positive number
        
        # calculate the max allowable annealing temp diff
        max_temp_diff = self.max_tdiff - self.min_tdiff
        
        # penalization if temp difference > max temp difference
        if(temp_diff > max_temp_diff):
            return (temp_diff - max_temp_diff)*self.penalty_tdiff
        
        # no penalization if temp difference <= max temp difference
        else:
            return 0
    
    '''The primer ought to be specific, i.e. only bind to one position on the dna sequence'''
    def cost_specificity(self, sq):
        
        # calculate the no of repeats of primer sequence in the dna sequence
        n_positions = self.func_count_specificity(sq)
        
        # penalization for every repeat
        return (n_positions-1)*self.penalty_specificity
        
    '''Runs of the same nucleotide in the primer should be avoided'''
    '''The maximum acceptable base run is usually 4'''
    def cost_runs(self, sq):
        
        # calculate the no of runs that exceed the acceptable threshold
        n_runs = self.func_count_runs(sq)
        
        # penalization for every run that exceeds the acceptable threshold
        return n_runs*self.penalty_runs
    
    '''Repeating sequences of two bases in the primer should be avoided'''
    def cost_repeats(self,sq):
        max_repeats = self.repeat_threshold
        
        # calculate the no of repeated two-bases in the primer
        n_repeats = self.func_count_repeats(sq)
        
        # penalization for every repeat that exceeds the acceptable threshold
        if n_repeats > max_repeats:
            return (n_repeats - max_repeats)*self.penalty_repeats
        
        # no penalization if n_repeats is within acceptable theshold
        else:
            return 0

In [13]:
'''Testing methods in Task 6'''

# primer sequences taken from eDimension test cases
DNA6 = PrimerDesign("DNA6")

# note that DNA6.dna_sequence contains 2 fp sequences and 1 rp sequence
DNA6.set_dna_sequence('atatgtatcgcattaaaaatacgaaaaaagtcatattcgcatcgcattaaaaatacgaaaaaagtcattatagcgatatatatattttatatataa')
fp = 'cattaaaaatacgaaaaaagtcat'
rp = 'atatatatattttatatataa'

'''Primer length is usually between 18 and 22 nucleotides'''
DNA6.max_length = 22
DNA6.min_length = 18

'''The difference in annealing temperatures between forward and reverse primers is usually less than 2°C'''
DNA6.max_tdiff = 2
DNA6.min_tdiff = 0

'''The CG content is usually between 40% and 60%'''
DNA6.max_cg = 0.6
DNA6.min_cg = 0.4

'''The primer annealing temperature is usually between 55°C and 63°C'''
DNA6.max_temp = 63
DNA6.min_temp = 55

'''The maximum acceptable base run is usually 4'''
DNA6.run_threshold = 4

'''Repeating sequences of two bases in the primer should be avoided'''
DNA6.repeat_threshold = 0

# Test Cases 1-2: check length cost function
assert DNA6.cost_length(fp) == 10*(24-22)
assert DNA6.cost_length(rp) == 0

# Test Case 3: check annealing temp cost function
assert DNA6.cost_temperature_difference(fp,rp) == 10*(abs(58-42)-2)

# Test Cases 4-5: check cgcontent cost function
assert DNA6.cost_cgcontent(fp) == 10*(0.4-5/24)
assert DNA6.cost_cgcontent(rp) == 10*(0.4-0)

# Test Cases 6-7: check annealing temp cost function
assert DNA6.cost_temperature(fp) == 0
assert DNA6.cost_temperature(rp) == 10*(55-42)

# Test Cases 8-9: check run cost function
assert DNA6.cost_runs(fp) == 10*2
assert DNA6.cost_runs(rp) == 0

# Test Cases 10-11: check repeat cost function
assert DNA6.cost_repeats(fp) == 0
assert DNA6.cost_repeats(rp) == 10*(12-0)

# Test Cases 12-13: check cost specificity
assert DNA6.cost_specificity(fp) == 10*(2-1)
assert DNA6.cost_specificity(rp) == 0

### Task 8

After choosing a pair of primers, all its properties are input into an objective function. This objective function cost_objective_function is the sum of all the cost functions described in Task 5, which outputs a score that is indicative of the closeness of these primers to the optimal primer. A score of zero means that the primers selected have met all the criteria. A score that is close to zero means that the primers selected is close to the optimal choice.

In [14]:
class PrimerDesign(PrimerDesign):
    
    def cost_objective_function(self, fp, rp):
        '''complete the calculation of the cost'''
        
        cost = self.cost_length(fp) + self.cost_length(rp) + self.cost_temperature(fp) + self.cost_temperature(rp) + \
        self.cost_cgcontent(fp) + self.cost_cgcontent(rp) + self.cost_temperature_difference(fp,rp) + \
        self.cost_specificity(fp) + self.cost_specificity(rp) + self.cost_runs(fp) + self.cost_runs(rp) + \
        self.cost_repeats(fp) + self.cost_repeats(rp)
        
        return cost

In [15]:
'''Testing method in Task 8'''

# primer sequences taken from eDimension test cases (same as Task 6)
# the output cost should be the sum of all the cost functions in Task 6

DNA8 = PrimerDesign("DNA8")

# note that DNA8.dna_sequence contains 2 fp sequences and 1 rp sequence
DNA8.set_dna_sequence('atatgtatcgcattaaaaatacgaaaaaagtcatattcgcatcgcattaaaaatacgaaaaaagtcattatagcgatatatatattttatatataa')
fp = 'cattaaaaatacgaaaaaagtcat'
rp = 'atatatatattttatatataa'

'''Primer length is usually between 18 and 22 nucleotides'''
DNA8.max_length = 22
DNA8.min_length = 18

'''The difference in annealing temperatures between forward and reverse primers is usually less than 2°C'''
DNA8.max_tdiff = 2
DNA8.min_tdiff = 0

'''The CG content is usually between 40% and 60%'''
DNA8.max_cg = 0.6
DNA8.min_cg = 0.4

'''The primer annealing temperature is usually between 55°C and 63°C'''
DNA8.max_temp = 63
DNA8.min_temp = 55

'''The maximum acceptable base run is usually 4'''
DNA8.run_threshold = 4

'''Repeating sequences of two bases in the primer should be avoided'''
DNA8.repeat_threshold = 0

assert DNA8.cost_objective_function(fp,rp) == 10*(24-22) + 0 + 10*(abs(58-42)-2) + 10*(0.4-5/24) + 10*(0.4-0) + 0 + 10*(55-42) + 10*2 + 0 + 0 + 10*(12-0) + 10*(2-1) + 0

### Task 9

The method cost_objective_function_info provides information on which criteria was met or not met. If the cost function for a particular criterion returns zero, the criterion is met. Note that in this output, the reverse primer displayed is the reverse complement of what is selected from the DNA sequence.

In [16]:
def checkCriterion(cost):
    if cost == 0:
        return "MET"
    else:
        return "NOT MET"

def ReverseComplement(rp):    
        
        # so far we have working with rp selected directly from the DNA sequence given
        # hence we need to reverse complement the input rp to output the correct rp
        new_rp = ""
        for base in rp:
            if base == 'a':
                new_rp += 't'
            elif base == 't':
                new_rp += 'a'
            elif base == 'c':
                new_rp += 'g'
            elif base == 'g':
                new_rp += 'c'
        
        return new_rp[::-1]

class PrimerDesign(PrimerDesign):
    
    def cost_objective_function_info(self, fp, rp):
        
        # obtain the correct rp
        correct_rp = ReverseComplement(rp)
        
        # forward and reverse primer info
        # note that for the rp, the cost function will still remain unchanged whether it is reverse complemented
        for i in [fp,rp]:
            if i == fp:
                print("===Forward Primer=== {}".format(fp))
            else:
                print("===Reverse Primer=== {}".format(correct_rp))
            print("Criterion{:>26}{:>25}{:>18}".format("Data", "Cost function score", "MET/ NOT MET"))
            print("------------------------------------------------------------------------------")
            print("length{:>29.3f}{:>25.3f}{:>18}".format(self.func_length(i), self.cost_length(i), checkCriterion(self.cost_length(i))))
            print("annealing temperature{:>14.3f}{:>25.3f}{:>18}".format(self.func_temperature(i), self.cost_temperature(i), checkCriterion(self.cost_temperature(i))))
            print("%cg_content{:>24.3f}{:>25.3f}{:>18}".format(self.func_cg_fraction(i), self.cost_cgcontent(i), checkCriterion(self.cost_cgcontent(i))))
            print("specificity{:>24.3f}{:>25.3f}{:>18}".format(self.func_count_specificity(i), self.cost_specificity(i), checkCriterion(self.cost_specificity(i))))
            print("runs{:>31.3f}{:>25.3f}{:>18}".format(self.func_count_runs(i), self.cost_runs(i), checkCriterion(self.cost_runs(i))))
            print("repeats{:>28.3f}{:>25.3f}{:>18}\n".format(self.func_count_repeats(i), self.cost_repeats(i), checkCriterion(self.cost_repeats(i))))
        
        # temperature difference between forward and reverse primers
        temp_diff = abs(self.func_temperature(fp) - self.func_temperature(rp))
        print("Temperature Difference{:>13.3f}{:>25.3f}{:>18}\n".format(temp_diff, self.cost_temperature_difference(fp,rp), checkCriterion(self.cost_temperature_difference(fp,rp))))
        
        # cost objective function
        cost_function = self.cost_objective_function(fp,rp)
        print("The calculated cost objective function value is {:.3f}".format(self.cost_objective_function(fp,rp)))
        

# Part 2 - Implement the Simulated Annealing Algorithm

### Task 12

Implement the simulated annealing algorithm to search for the primers that give the best possible score.

In [17]:
class PrimerDesign(PrimerDesign): 

    def func_simulated_annealing(self):
        
        # step 1.1: deciding simulation parameters
        temperature = self.initial_temperature
        stopping_temperature = self.stopping_temperature
        drop = self.drop_fraction
        
        # step 1.2: choosing a possible solution
        fp_current = self.func_select_random(sqtype='forward')
        rp_current = self.func_select_random(sqtype='reverse')
        current_cost = self.cost_objective_function(fp_current, rp_current)
        
        iteration = 0
        while(temperature > stopping_temperature):
            # step 2: obtain a new possible solution
            fp_new = self.get_primer_neighbour(fp_current)
            rp_new = self.get_primer_neighbour(rp_current)
            new_cost = self.cost_objective_function(fp_new, rp_new)
            
            # step 3: accept new soln if it is a lower cost than the current soln
            if(new_cost < current_cost):
                fp_current = fp_new
                rp_current = rp_new
                current_cost = new_cost
            
            # step 4: if the current soln is worse, we may still accept it
            else:
                # step 4a: calculate an acceptance probability
                delta = new_cost - current_cost
                acceptance_probability = math.exp(-delta/temperature)
                
                # step 4b: throw a dice
                num = random.uniform(0,1)
                
                # step 4c: accept the new soln if num < acceptance probability
                if(acceptance_probability > num):
                    fp_current = fp_new
                    rp_current = rp_new
                    current_cost = new_cost
                    
                # step 5: decrease the temp by a small factor
                temperature = temperature*drop
            iteration += 1
        
        self.cost_objective_function_info(fp_current, rp_current)
        print("The number of iterations required is {}.".format(iteration))
        
        # check primer locations
        if fp_current not in self.dna_sequence or rp_current not in self.dna_sequence:
            return "Error" # primer not found
        else:
            print("The forward primer location is from {} to {}.".format(self.dna_sequence.index(fp_current), self.dna_sequence.index(fp_current) + len(fp_current)))
            print("The reverse primer location is from {} to {}.".format(self.dna_sequence.index(rp_current), self.dna_sequence.index(rp_current) + len(rp_current)))
        
        # expected PCR length
        print("The expcted PCR length should be {} nucleotides long.".format(self.dna_sequence.index(rp_current) + len(rp_current) - self.dna_sequence.index(fp_current) + 1))
        
    def get_primer_neighbour(self, sq):
        new_start = self.dna_sequence.index(sq) + random.randint(-10,10)
        new_primer_length = random.randint(self.min_length, self.max_length)
        new_primer = self.dna_sequence[new_start: new_start + new_primer_length + 1]
        return new_primer


### Store the DNA sequence given to you in the variable below 

In [18]:
dna_sequence = '''1 gattggctgg cgcggattcc agctgctttc caagtcagcg gcgcctagtg agagtcaggg
61 gggcccggcc cgcgccctcc ccgcccagcc gcctccccgt cgacgcccag ggctggggcg
121 agccaggctg cctttcgaac ttggggggct tctcctcttg tctcccactg gtgctctggc
181 tgtgaatcca tccaggggcc aggatgacaa tccgacacca aggccagcag tacaggccga
241 ggatggcatt tctccaaaag attgaagcgc tcgtgaagga catgcagaac ccagagacag
301 gggtccgaat gcagaaccag agggtcctgg tcaccagcgt tcctcatgcc atgacaggaa
361 gtgatgttct gcaatggatc gtccagcggc tttggatctc cagtctggag gcacagaact
421 tgggcaactt tattgtcagg tatggctaca tttaccccct gcaagacccc aagaatctca
481 ttctcaagcc tgatggcagc ctctacagat ttcagacacc gtatttctgg cccacccagc
541 agtggccagc tgaagatacc gattacgcca tctatctggc caagcgaaat atcaaaaaga
601 aagggatttt ggaagaatat gaaaaggaaa attacaattt cttgaaccaa aaaatgaact
661 ataagtggga ctttgtcatt atgcaggcca aagagcagta cagggctgga aaggagagga
721 acaaagcaga cagatatgcc ctggactgcc aggagaaggc atactggctg gtgcaccgat
781 gccctcctgg aatggacaat gtgctggact acggcctgga ccgagtgacc aatccgaatg
841 aagtcaaggt aaaccagaaa caaacagtcg ttgctgtcaa aaaagagatc atgtattacc
901 aacaggcctt gatgaggtcc acagtgaagt cttctgtgtc cctgggaggg attgtgaaat
961 acagtgagca gttctcatcc aacgatgcca tcatgtcagg ctgcctcccc agcaacccct
1021 ggatcaccga tgacacccag ttctgggact taaatgccaa attggtggaa atcccaacca
1081 agatgcgagt ggaacgatgg gccttcaact tcagcgaatt gatccgagac cccaaaggtc
1141 gacagagctt ccagtacttc ctcaagaaag ggctcagtgg agagaatctg ggattctggg
1201 aagcctgcga ggatctgaag tatggagatc agtccaaagt caaggagaaa gcagaggaga
1261 tttacaagct gttcctggcc ccgggggcga ggcgctggat caacatagat ggcaaaacca
1321 tggacatcac agtgaagggg ctgaagcacc cccaccgcta tgtgctggac gccgcacaaa
1381 cccacattta catgctcatg aagaaggatt cttatgctcg ctatttaaaa tctccgatct
1441 ataaggacat gctggccaaa gctattgaac ctcaggaaac caccaagaaa agctccaccc
1501 tcccttttat gcggcgtcac ctgcgctcca gcccaagccc tgtcatcctg agacagctgg
1561 aagaggaagc caaggcccga gaagcagcca acactgtgga catcacccag ccgggccagc
1621 acatggctcc cagcccccat ctgaccgtgt acaccgggac ctggaggccc ccgtctcctt
1681 ctagcccctt ctcctcctcc tgccgctccc ccaggaagcc tttcgcctca cccagccgct
1741 tcatccggcg acccagcacc accatctgcc cctcacccat cagagtggcc ttggagagct
1801 catcgggctt ggagcagaaa ggggagtgca gcgggtccat ggccccccgt gggccctctg
1861 tcaccgagag cagcgaggcc tccctcgaca cctcctggcc tcgcagccgg cccagggccc
1921 ctcctaaggc ccgcatggct ctgtccttca gcaggtttct gagacgaggc tgtctggcct
1981 cacctgtctt tgccaggctc tcacccaagt gccctgctgt gtcccacggg agggtgcagc
2041 ccctggggga cgtgggccag cagctgccac gattgaaatc caagagagta gcaaactttt
2101 tccagatcaa aatggatgtg cccacgggga gcgggacctg cttgatggac tcggaggatg
2161 ctggaacagg agagtcgggt gaccgggcca cagaaaagga ggtcatctgc ccctgggaga
2221 gcctgtaagg aaagaggcag gctgagctgg gggctctgga ccaggaagat gctctgacag
2281 atgccatggt atgggccaca ggacacactt gctcgagaac caaagtgcat ttgggtgaca
2341 tttgaagatt ggggagacaa gatggggtag attgtggcaa agaatgctct ggctggttac
2401 caggggccaa ctccttctcc tcttcctgac cctccctccc ctgggcagaa gaaacgcatg
2461 tggaccagaa gactttccct gctgccttaa aaccaataaa aggttaactt taagtttctt
2521 ggaaaaaaaa aaaaaaaaaa
'''

### Instantiate your class and read in the DNA sequence

In [19]:
DNA_test = PrimerDesign("DNA_test")
DNA_test.set_dna_sequence(dna_sequence)

### If you need to adjust any parameter from their default values in the init method, do it here

In [20]:
'''parameters for the length criterion'''
'''Primer length is usually between 18 and 22 nucleotides'''
# This length is long enough for adequate specificity
# and short enough for primers to bind easily to the template at the annealing temperature.
DNA_test.max_length = 22
DNA_test.min_length = 18
DNA_test.penalty_length = 10

'''parameters for the temperature difference criterion'''
'''The difference in annealing temperatures between forward and reverse primers is usually less than 2°C.'''
# This is because during the PCR annealing step, there is only one annealing temperature where both the reverse and
# forward primers are expected to anneal to the denatured single DNA strand, so the difference cannot be too far apart
# if not the DNA amplification by Taq polymerase during the elongation step cannot happen
DNA_test.max_tdiff = 2
DNA_test.min_tdiff = 0
# higher weightage because of the reason described above
DNA_test.penalty_tdiff = 30

'''parameters for the cg content criterion'''
'''The CG content is usually between 40% and 60%'''
DNA_test.max_cg = 0.6
DNA_test.min_cg = 0.4
# higher weightage so that the order of the magnitude of the score is comparable to the rest
DNA_test.penalty_cg = 100 

'''parameters for the annealing temperature criterion'''
'''The primer annealing temperature is usually between 55°C and 63°C'''
DNA_test.max_temp = 63
DNA_test.min_temp = 55
# less weightage because the annealing temp is calculated using the cg_fraction
# so the penalty would have been imposed on the cg_content cost function
DNA_test.penalty_temp = 10

'''parameters for the run criterion'''
'''The maximum acceptable base run is usually 4'''
DNA_test.run_threshold = 4
# less weightage because we assume the other sequences
# to be sufficiently different for the primer to recognise and bind
DNA_test.penalty_runs = 10

'''parameters for the repeat criterion'''
'''Repeating sequences of two bases in the primer should be avoided'''
DNA_test.repeat_threshold = 0
# less weightage because we have already made the requirement stricter
# by setting the threshold to be as low as possible i.e. 0
DNA_test.penalty_repeats = 10

'''parameters for the specificity criterion'''
# higher weightage because we want the chosen primer to bind at a specific location
# if not the PCR product size will be different and RFLP gel electrophoresis will be hard
DNA_test.penalty_specificity = 20 

# our dna sequence shows a mutated sequence at the 1100th position
# so we design our forward and reverse primers should be at least 300bp before and after the identified region
# so that the final PCR product is around 300 - 1000 bp
# gene sequencing and identification can then be performed on this length
# this is because 100 codons are needed as the shortest length of a putative gene, so at least 300bp
# also, the PCR length should be sufficiently long to be run and easily identified on the gel
# after performing RFLP enzyme digestion and gel electrophoresis

'''locations where the forward primer should be chosen from'''
DNA_test.fp_start = 600
DNA_test.fp_end = 700

'''locations where the reverse primer should be chosen from'''
DNA_test.rp_start = 1600
DNA_test.rp_end = 1700

### Show the outcome of your testing and the functions in the subsequent cells 

In [21]:
'''Testing method for Task 9'''

fp = DNA_test.func_select_random()
rp = DNA_test.func_select_random(sqtype='reverse')
DNA_test.cost_objective_function_info(fp,rp)

===Forward Primer=== aatttcttgaaccaaaaaat
Criterion                      Data      Cost function score      MET/ NOT MET
------------------------------------------------------------------------------
length                       20.000                    0.000               MET
annealing temperature        48.000                   70.000           NOT MET
%cg_content                   0.200                   20.000           NOT MET
specificity                   1.000                    0.000               MET
runs                          1.000                   10.000           NOT MET
repeats                       0.000                    0.000               MET

===Reverse Primer=== ctgggagccatgtgctggcc
Criterion                      Data      Cost function score      MET/ NOT MET
------------------------------------------------------------------------------
length                       20.000                    0.000               MET
annealing temperature        68.000           

In [22]:
'''Testing method for Task 10'''

DNA_test.func_simulated_annealing()

===Forward Primer=== gagcagtacagggctggaaa
Criterion                      Data      Cost function score      MET/ NOT MET
------------------------------------------------------------------------------
length                       20.000                    0.000               MET
annealing temperature        62.000                    0.000               MET
%cg_content                   0.550                    0.000               MET
specificity                   1.000                    0.000               MET
runs                          0.000                    0.000               MET
repeats                       0.000                    0.000               MET

===Reverse Primer=== aggagaaggggctagaagg
Criterion                      Data      Cost function score      MET/ NOT MET
------------------------------------------------------------------------------
length                       19.000                    0.000               MET
annealing temperature        60.000            

### What were the simulation parameters that you used (in Step 1.1 of the algorithm)?

self.initial_temperature = 200

self.stopping_temperature = 0.01

self.drop_fraction = 0.999
        
### From the output of your code, how well does the final solution meet all the criteria?

The final solution meets all the criteria very well as the individual cost function score is 0.000 for all the criterion.

### How many iterations were required to obtain this solution?

10325 iterations were required to obtain this solution.

### Will a brute-force algorithm (one that considers every single possible solution) require fewer or more iterations? Explain.

A brute-force algorithm will require more iterations. As a rough calculation for the brute force algorithm using the 2.5kbp long DNA sequence provided, assuming that we begin by displacing the forward and reverse primers from the halfway point, conservatively we need at least 1000^1000 iterations, which is much more than the simulated annealing algorithm.

The simulated annealing algorithm checks the cost function score against certain criteria, so is able to very quickly approach the optimal solution.